In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!cp -r '/content/gdrive/My Drive/MURA-v1.1.zip' MURA-v1.1.zip
!unzip MURA-v1.1.zip

Output hidden; open in https://colab.research.google.com to view.

In [0]:

import io
import requests
from PIL import Image
from torchvision import models, transforms
from torch.autograd import Variable
import pdb
import random
import csv
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import os
from PIL import Image
import torchvision.models as models
%matplotlib inline
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [0]:
class MuraBodyDataset(torch.utils.data.Dataset):
    
    def __init__(self,text_file,root_dir, transform ): #Normalize to pretrained VGGNet
        """
        Args:
            text_file(string): path to text file
            root_dir(string): directory with all train images
        """
        #File with the Path
        self.name_frame = pd.read_csv(text_file,sep=",",usecols=[0],dtype = 'str')
        #File with labels
        self.label_frame = pd.read_csv(text_file,sep=",", usecols=[1])
        self.root_dir = root_dir
        self.transform = transform
                                       
    def __len__(self):
        return len(self.name_frame)

    def __getitem__(self, idx):
        #Pull image
        img_name = os.path.join(self.root_dir, self.name_frame.iloc[idx, 0])
        if not os.path.exists(img_name):
          print(img_name)
        else:        
          #Apply adapative threshold to highlight key features in the image   
          image = cv2.imread(img_name,0)
          image = cv2.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,11,2)  
          image = Image.fromarray(image)
          image = image.convert('L')

          #Apply tensor transformations to images
          image = self.transform(image) 

          # I included labels in the train_image_path file in excel using a find("positive")
          labels = self.label_frame.iloc[idx,0] 

          #Formatting for labels
          labels = np.array(labels)
          labels = np.reshape(labels, (1,1))
          labels= torch.from_numpy(labels.astype('int'))

          sample = {'image': image, 'labels': labels}
        
        return sample

In [0]:
MuraTrainSet = MuraBodyDataset(text_file ='/content/MURA-v1.1/Forearm_Mura.csv',
                               root_dir = '/content/',
                               transform = transforms.Compose([transforms.Resize((224,224)),
                                                          transforms.ToTensor(),
                                                          transforms.Normalize(
                                                              mean=[0.456],
                                                              std=[0.224])]))

MuraTrainLoader = torch.utils.data.DataLoader(MuraTrainSet,batch_size=64,shuffle=True, num_workers=0)

MuraTestSet = MuraBodyDataset(text_file ='/content/MURA-v1.1/Forearm_Mura_test.csv',
                              root_dir = '/content/',
                              transform = transforms.Compose([transforms.Resize((224,224)),
                                                          transforms.ToTensor(),
                                                          transforms.Normalize(
                                                              mean=[0.456],
                                                              std= [0.225])]))

MuraTestLoader = torch.utils.data.DataLoader(MuraTestSet,batch_size=64,shuffle=False, num_workers=0)

In [0]:
dense_net169 = models.densenet169(pretrained=True)

for param in dense_net169.features.parameters():
    param.requires_grad = False
    
dense_net169.features._modules['conv0']  = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
#Adjust to grayscale = 1 channel. Requires gradient is True by default
dense_net169.classifier = nn.Linear(in_features=1664, out_features=1, bias=True)



In [0]:
def train(epoch, max_batch, loss_array, accuracy_array, learning_rate, weight_decay):
    dense_net169.train()
    train_loss = 0
    TN = 0
    TP = 0
    FP = 0
    FN = 0
    for i, sample_batched in enumerate(MuraTrainLoader,1):
        if i <max_batch+1:
        
            inputs = sample_batched['image']
            labels = sample_batched['labels']
            inputs, labels = inputs.to(device), labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = dense_net169(inputs)
            labels = labels.view(len(sample_batched['labels']),-1)
            loss = criterion(outputs.float(), labels.float())
            loss.backward()
            optimizer.step()
            train_loss+= loss.item()

            pred = (torch.sigmoid(outputs).data > 0.5).int()
            labels = labels.int()
            TP += ((pred == 1) & (labels == 1) ).float().sum() #True Positive Count
            TN += ((pred == 0) & (labels == 0) ).float().sum() #True Negative Count
            FP += ((pred == 1) & (labels == 0) ).float().sum() #False Positive Count
            FN += ((pred == 0) & (labels == 1) ).float().sum() #False Negative Count
        else:
            #print('Batches completed: {}'.format(max_batch))
            #max_batch=max_batch+max_batch
            break            
    TP = TP.cpu().numpy()
    TN = TN.cpu().numpy()
    FP = FP.cpu().numpy()
    FN = FN.cpu().numpy()
    accuracy = (TP + TN)/(TP + TN + FP + FN)
    precision = TP/(TP + FP)
    recall = TP/(TP + FN)
    f1_score = 2 * (precision*recall)/(precision+recall)
    p_yes = ((TP + FP)/(TP + TN + FP + FN))*((TP + FN)/(TP + TN + FP + FN))
    p_no = ((TN + FN)/(TP + TN + FP + FN))*((FP + TN)/(TP + TN + FP + FN))
    p_e = p_yes+p_no
    kappa = (accuracy - p_e)/(1-p_e)
    accuracy_array.append([learning_rate, weight_decay, epoch, TP, TN, FP, FN, accuracy, precision, recall, f1_score])
    train_loss = float(train_loss)/float(i)
    loss_array.append([learning_rate, weight_decay, epoch, train_loss])
    # print statistics
    if epoch % 5==0:
      print('Train Epoch:{}  Accuracy: {:.2f}   Average Loss: {:.2f}  Precision: {:.2f}   F1 Score: {:.2f} Kappa: {:.2f}\n'.
            format(epoch, accuracy, train_loss, precision, f1_score, kappa))


In [0]:
def hook_feature(module, input, output):
    features_blobs.append(output.data.cpu().numpy())
    
def returnCAM(feature_conv, weight_softmax, class_idx):
    # generate the class activation maps upsample to 256x256
    size_upsample = (256, 256)
    bz, nc, h, w = feature_conv.shape
    output_cam = []
    #print(class_idx)
    for idx in class_idx:
        #print(idx)
        cam = weight_softmax.dot(feature_conv.reshape((nc, h*w)))
        cam = cam.reshape(h, w)
        cam = cam - np.min(cam)
        cam_img = cam / np.max(cam)
        cam_img = np.uint8(255 * cam_img)
        output_cam.append(cv2.resize(cam_img, size_upsample))
    return output_cam
  
normalize = transforms.Normalize(
    mean=[0.456],
    std= [0.225]
   #mean=[0.485, 0.456, 0.406],
   #std=[0.229, 0.224, 0.225]
)
preprocess = transforms.Compose([
   transforms.Resize((224,224)),
   transforms.ToTensor(),
   normalize
])

In [0]:
dense_train_loss = []
dense_train_accuracy =[]



In [0]:
dense_train_loss = dense_train_loss
dense_train_accuracy = dense_train_accuracy
Bodypart = 'Forearm'
Path = '/content/gdrive/My Drive/MURA-v1.1'
os.path.join(Path,Bodypart)
import gc


IMG_URLS = ['/content/MURA-v1.1/train/XR_FOREARM/patient09089/study2_negative/image1.png',
            '/content/MURA-v1.1/train/XR_FOREARM/patient09270/study2_negative/image2.png',
            '/content/MURA-v1.1/train/XR_FOREARM/patient09083/study1_positive/image1.png',
            '/content/MURA-v1.1/train/XR_FOREARM/patient09083/study1_positive/image2.png',
            '/content/MURA-v1.1/train/XR_FOREARM/patient05601/study1_positive/image1.png']


for i in range(1,17):
  #reset model to original
  dense_net169 = models.densenet169(pretrained=True)

  for param in dense_net169.features.parameters():
      param.requires_grad = False

  dense_net169.features._modules['conv0']  = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

  #Adjust to grayscale = 1 channel. Requires gradient is True by default
  dense_net169.classifier = nn.Linear(in_features=1664, out_features=1, bias=True)

  dense_net169.to(device)

  learning = round(np.exp(random.uniform(np.log(.0001), np.log(.01))),4) #pull geometrically
  w = round(np.exp(random.uniform(np.log(3.1e-7), np.log(3.1e-5))),10) #pull geometrically


  criterion = nn.BCEWithLogitsLoss() 
  optimizer = optim.Adam(dense_net169.parameters(), lr=learning, weight_decay = w)
  
  Model = 'Model'+str(i)
  
  for epoch in range(1,21): 
      train(epoch,10000, dense_train_loss,dense_train_accuracy, learning, w)
      if epoch % 5 == 0:
        net = dense_net169.to(device)
        finalconv_name = 'features'

        net.eval()

        # hook the feature extractor
        features_blobs = []

        net._modules.get(finalconv_name).register_forward_hook(hook_feature)

        # get the softmax weight
        params = list(net.parameters())
        weight_softmax = np.squeeze(params[-2].data.cpu().numpy())
        
        for j in range(len(IMG_URLS)):
          IMG_URL = IMG_URLS[j]

          img_pil = Image.open(IMG_URL)
          img_pil = img_pil.convert(mode='L')
          image_path = os.path.join(Path,Bodypart,Model,'Images','test.jpg')
          img_pil.save(image_path)

          img_tensor = preprocess(img_pil)
          img_variable = Variable(img_tensor.unsqueeze(0))
          logit = net(img_variable.cuda())
          h_x = torch.sigmoid(logit).data.squeeze()
          probs, idx = h_x.sort(0, True)
          probs = np.array([probs.cpu().numpy()])
          idx = np.array([idx.cpu().numpy()])
          #print([probs])
          #print([idx])

          CAMs = returnCAM(features_blobs[0], weight_softmax, [idx[0]])

          # render the CAM and output
          #print('output CAM.jpg for the top1 prediction: %s'%classes[idx[0]])
          img = cv2.imread(image_path,0)
          img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,11,2)
          img = np.float32(cv2.resize(img, (224, 224)))/2
          img = np.reshape(img,(224,224,1))
          height, width,_ = img.shape
          heatmap = cv2.applyColorMap(cv2.resize(CAMs[0],(width, height)), cv2.COLORMAP_WINTER)
          result = heatmap * 0.95 + img * 0.5
          image_num = 'CAM'+ str(j)+'_'+str(epoch)+'.jpg'
          #print(image_num)
          image_path = os.path.join(Path,Bodypart,Model,'Images',image_num)
          cv2.imwrite(image_path, result)
          gc.collect()
        
  Model_name = Model + '.pth'
  Model_Path = os.path.join(Path,Bodypart,Model,Model_name)
  torch.save(dense_net169.state_dict(), Model_Path)
  print('Model Completed: ', Model)

  dense_train_loss_df = pd.DataFrame(dense_train_loss,
                                    columns =['learning_rate', 'weight_decay', 'epoch', 'train_loss'] )
  dense_train_accuracy_df = pd.DataFrame(dense_train_accuracy,
                                        columns = ['learning_rate', 'weight_decay', 'epoch', 'TP', 'TN', 'FP', 'FN',
                                                   'accuracy', 'precision', 'recall', 'f1_score'])

  dense_train_loss_df.to_csv(os.path.join(Path,Bodypart,'loss.csv'))
  dense_train_accuracy_df.to_csv(os.path.join(Path,Bodypart,'accuracy.csv'))

    

Train Epoch:5  Accuracy: 0.76   Average Loss: 0.51  Precision: 0.75   F1 Score: 0.61 Kappa: 0.45

Train Epoch:10  Accuracy: 0.81   Average Loss: 0.41  Precision: 0.81   F1 Score: 0.71 Kappa: 0.57

Train Epoch:15  Accuracy: 0.86   Average Loss: 0.35  Precision: 0.84   F1 Score: 0.79 Kappa: 0.68

Train Epoch:20  Accuracy: 0.89   Average Loss: 0.28  Precision: 0.87   F1 Score: 0.85 Kappa: 0.76

Model Completed:  Model1
Train Epoch:5  Accuracy: 0.75   Average Loss: 0.52  Precision: 0.71   F1 Score: 0.60 Kappa: 0.42

Train Epoch:10  Accuracy: 0.83   Average Loss: 0.40  Precision: 0.82   F1 Score: 0.75 Kappa: 0.62

Train Epoch:15  Accuracy: 0.87   Average Loss: 0.31  Precision: 0.86   F1 Score: 0.81 Kappa: 0.71

Train Epoch:20  Accuracy: 0.90   Average Loss: 0.26  Precision: 0.89   F1 Score: 0.86 Kappa: 0.78

Model Completed:  Model2
Train Epoch:5  Accuracy: 0.77   Average Loss: 0.50  Precision: 0.77   F1 Score: 0.62 Kappa: 0.47

Train Epoch:10  Accuracy: 0.81   Average Loss: 0.42  Precision

In [0]:
PATH = '/content/gdrive/My Drive/MURA-v1.1/Forearm/Model2/Model2.pth'
dense_net169.load_state_dict(torch.load(PATH))


In [0]:
def test(epoch, loss_array, accuracy_array, pred_array):
    #Have our model in evaluation mode
    dense_net169.eval()
    #Set losses and Correct labels to zero
    test_loss = 0
    TN = 0
    TP = 0
    FP = 0
    FN = 0
    with torch.no_grad():
        for i, sample_batched in enumerate(MuraTestLoader,1):
            inputs = sample_batched['image']
            labels = sample_batched['labels']
            #body_part = sample_batched['labels']
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = dense_net169(inputs)
            labels = labels.view(len(sample_batched['labels']),-1)
            loss = criterion(outputs.float(), labels.float())
            test_loss += loss.item()

            pred = (torch.sigmoid(outputs).data > 0.5).int()
            pred_array.append([labels, pred])
            labels = labels.int()
            TP += ((pred == 1) & (labels == 1) ).float().sum() #True Positive Count
            TN += ((pred == 0) & (labels == 0) ).float().sum() #True Negative Count
            FP += ((pred == 1) & (labels == 0) ).float().sum() #False Positive Count
            FN += ((pred == 0) & (labels == 1) ).float().sum() #False Negative Count
            #print('TP: {}\t TN: {}\t FP: {}\t FN: {}\n'.format(TP,TN,FP,FN) )    

        accuracy = (TP + TN)/(TP + TN + FP + FN)
        precision = TP/(TP + FP)
        recall = TP/(TP + FN)
        f1_score = 2 * (precision*recall)/(precision+recall)
        accuracy_array.append([TP, TN, FP, FN, accuracy, precision, recall, f1_score])
        test_loss = float(test_loss)/float(i)
        loss_array.append(test_loss)
        # print statistics        
        print('Test  Epoch:{}  Accuracy: {:.2f}   Average Loss: {:.2f}  Precision: {:.2f}   F1 Score: {:.2f}\n'.
              format(epoch, accuracy, test_loss, precision, f1_score))

In [0]:
for i in range(1,17):
  dense_net169 = models.densenet169(pretrained=True)
  dense_net169.features._modules['conv0']  = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  dense_net169.classifier = nn.Linear(in_features=1664, out_features=1, bias=True)
  criterion = nn.BCEWithLogitsLoss() 
  #optimizer = optim.Adam(dense_net169.parameters(), lr=learning, weight_decay = w)
  
  dense_net169.to(device)

  PATH = '/content/gdrive/My Drive/MURA-v1.1/Forearm/Model'+str(i)+'/Model'+str(i)+'.pth'
  dense_net169.load_state_dict(torch.load(PATH))

  dense_test_loss = []
  dense_test_accuracy = []
  dense_test_pred =[]

  test(i, dense_test_loss,dense_test_accuracy, dense_test_pred )


Test  Epoch:1  Accuracy: 0.60   Average Loss: 0.87  Precision: 0.73   F1 Score: 0.44

Test  Epoch:2  Accuracy: 0.68   Average Loss: 0.77  Precision: 0.76   F1 Score: 0.62

Test  Epoch:3  Accuracy: 0.69   Average Loss: 0.74  Precision: 0.74   F1 Score: 0.66

Test  Epoch:4  Accuracy: 0.62   Average Loss: 0.76  Precision: 0.65   F1 Score: 0.58

Test  Epoch:5  Accuracy: 0.59   Average Loss: 0.94  Precision: 0.69   F1 Score: 0.45

Test  Epoch:6  Accuracy: 0.62   Average Loss: 0.91  Precision: 0.67   F1 Score: 0.55

Test  Epoch:7  Accuracy: 0.61   Average Loss: 0.79  Precision: 0.71   F1 Score: 0.48

Test  Epoch:8  Accuracy: 0.64   Average Loss: 0.92  Precision: 0.76   F1 Score: 0.53

Test  Epoch:9  Accuracy: 0.60   Average Loss: 0.67  Precision: 0.77   F1 Score: 0.42

Test  Epoch:10  Accuracy: 0.66   Average Loss: 0.76  Precision: 0.69   F1 Score: 0.63

Test  Epoch:11  Accuracy: 0.66   Average Loss: 0.80  Precision: 0.82   F1 Score: 0.54

Test  Epoch:12  Accuracy: 0.67   Average Loss: 0.79 

** Humerus Data**

In [0]:
MuraTrainSet = MuraBodyDataset(text_file ='/content/MURA-v1.1/Humerus_Mura.csv',
                               root_dir = '/content/',
                               transform = transforms.Compose([transforms.Resize((224,224)),
                                                          transforms.ToTensor(),
                                                          transforms.Normalize(
                                                              mean=[0.456],
                                                              std=[0.224])]))

MuraTrainLoader = torch.utils.data.DataLoader(MuraTrainSet,batch_size=32,shuffle=True, num_workers=0)

MuraTestSet = MuraBodyDataset(text_file ='/content/MURA-v1.1/Humerus_Mura_test.csv',
                              root_dir = '/content/',
                              transform = transforms.Compose([transforms.Resize((224,224)),
                                                          transforms.ToTensor(),
                                                          transforms.Normalize(
                                                              mean=[0.456],
                                                              std= [0.225])]))

MuraTestLoader = torch.utils.data.DataLoader(MuraTestSet,batch_size=32,shuffle=False, num_workers=0)

In [0]:
dense_train_loss = dense_train_loss
dense_train_accuracy = dense_train_accuracy
Bodypart = 'Humerus'
Path = '/content/gdrive/My Drive/MURA-v1.1'
os.path.join(Path,Bodypart)
import gc


IMG_URLS = ['/content/MURA-v1.1/train/XR_HUMERUS/patient02938/study1_positive/image2.png',
            '/content/MURA-v1.1/train/XR_HUMERUS/patient02939/study1_positive/image1.png',
            '/content/MURA-v1.1/train/XR_HUMERUS/patient02939/study1_positive/image2.png',
            '/content/MURA-v1.1/train/XR_HUMERUS/patient02940/study1_negative/image1.png',
            '/content/MURA-v1.1/train/XR_HUMERUS/patient02940/study1_negative/image2.png',
            '/content/MURA-v1.1/train/XR_HUMERUS/patient02941/study1_negative/image1.png']


for i in range(1,17):
  #reset model to original
  dense_net169 = models.densenet169(pretrained=True)

  for param in dense_net169.features.parameters():
      param.requires_grad = False

  dense_net169.features._modules['conv0']  = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

  #Adjust to grayscale = 1 channel. Requires gradient is True by default
  dense_net169.classifier = nn.Linear(in_features=1664, out_features=1, bias=True)

  dense_net169.to(device)

  learning = round(np.exp(random.uniform(np.log(.0001), np.log(.01))),4) #pull geometrically
  w = round(np.exp(random.uniform(np.log(3.1e-7), np.log(3.1e-5))),10) #pull geometrically


  criterion = nn.BCEWithLogitsLoss() 
  optimizer = optim.Adam(dense_net169.parameters(), lr=learning, weight_decay = w)
  
  Model = 'Model'+str(i)
  
  for epoch in range(1,21): 
      train(epoch,10000, dense_train_loss,dense_train_accuracy, learning, w)
      if epoch % 5 == 0:
        net = dense_net169.to(device)
        finalconv_name = 'features'

        net.eval()

        # hook the feature extractor
        features_blobs = []

        net._modules.get(finalconv_name).register_forward_hook(hook_feature)

        # get the softmax weight
        params = list(net.parameters())
        weight_softmax = np.squeeze(params[-2].data.cpu().numpy())
        
        for j in range(len(IMG_URLS)):
          IMG_URL = IMG_URLS[j]

          img_pil = Image.open(IMG_URL)
          img_pil = img_pil.convert(mode='L')
          image_path = os.path.join(Path,Bodypart,Model,'Images','test.jpg')
          img_pil.save(image_path)

          img_tensor = preprocess(img_pil)
          img_variable = Variable(img_tensor.unsqueeze(0))
          logit = net(img_variable.cuda())
          h_x = torch.sigmoid(logit).data.squeeze()
          probs, idx = h_x.sort(0, True)
          probs = np.array([probs.cpu().numpy()])
          idx = np.array([idx.cpu().numpy()])
          #print([probs])
          #print([idx])

          CAMs = returnCAM(features_blobs[0], weight_softmax, [idx[0]])

          # render the CAM and output
          #print('output CAM.jpg for the top1 prediction: %s'%classes[idx[0]])
          img = cv2.imread(image_path,0)
          img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,11,2)
          img = np.float32(cv2.resize(img, (224, 224)))/2
          img = np.reshape(img,(224,224,1))
          height, width,_ = img.shape
          heatmap = cv2.applyColorMap(cv2.resize(CAMs[0],(width, height)), cv2.COLORMAP_WINTER)
          result = heatmap * 0.95 + img * 0.5
          image_num = 'CAM'+ str(j)+'_'+str(epoch)+'.jpg'
          #print(image_num)
          image_path = os.path.join(Path,Bodypart,Model,'Images',image_num)
          cv2.imwrite(image_path, result)
          gc.collect()
        
  Model_name = Model + '.pth'
  Model_Path = os.path.join(Path,Bodypart,Model,Model_name)
  torch.save(dense_net169.state_dict(), Model_Path)
  print('Model Completed: ', Model)

  dense_train_loss_df = pd.DataFrame(dense_train_loss,
                                    columns =['learning_rate', 'weight_decay', 'epoch', 'train_loss'] )
  dense_train_accuracy_df = pd.DataFrame(dense_train_accuracy,
                                        columns = ['learning_rate', 'weight_decay', 'epoch', 'TP', 'TN', 'FP', 'FN',
                                                   'accuracy', 'precision', 'recall', 'f1_score'])

  dense_train_loss_df.to_csv(os.path.join(Path,Bodypart,'loss.csv'))
  dense_train_accuracy_df.to_csv(os.path.join(Path,Bodypart,'accuracy.csv'))

    
    

Train Epoch:5  Accuracy: 0.63   Average Loss: 0.65  Precision: 0.69   F1 Score: 0.51 Kappa: 0.25

Train Epoch:10  Accuracy: 0.69   Average Loss: 0.62  Precision: 0.73   F1 Score: 0.63 Kappa: 0.38

Train Epoch:15  Accuracy: 0.73   Average Loss: 0.59  Precision: 0.75   F1 Score: 0.70 Kappa: 0.46

Train Epoch:20  Accuracy: 0.74   Average Loss: 0.57  Precision: 0.79   F1 Score: 0.69 Kappa: 0.47

Model Completed:  Model1
Train Epoch:5  Accuracy: 0.68   Average Loss: 0.62  Precision: 0.71   F1 Score: 0.62 Kappa: 0.36

Train Epoch:10  Accuracy: 0.76   Average Loss: 0.55  Precision: 0.77   F1 Score: 0.73 Kappa: 0.51

Train Epoch:15  Accuracy: 0.78   Average Loss: 0.51  Precision: 0.82   F1 Score: 0.75 Kappa: 0.56

Train Epoch:20  Accuracy: 0.83   Average Loss: 0.46  Precision: 0.83   F1 Score: 0.82 Kappa: 0.66

Model Completed:  Model2
Train Epoch:5  Accuracy: 0.68   Average Loss: 0.62  Precision: 0.67   F1 Score: 0.66 Kappa: 0.36

Train Epoch:10  Accuracy: 0.83   Average Loss: 0.40  Precision

In [0]:
dense_test_loss = []
dense_test_accuracy = []
dense_test_pred =[]
for epoch in range(1,2): 
    test(epoch, dense_test_loss,dense_test_accuracy, dense_test_pred )

Test  Epoch:1  Accuracy: 0.62   Average Loss: 0.66  Precision: 0.69   F1 Score: 0.51



**Elbow Data**

In [0]:
MuraTrainSet = MuraBodyDataset(text_file ='/content/MURA-v1.1/Elbow_Mura.csv',
                               root_dir = '/content/',
                               transform = transforms.Compose([transforms.Resize((224,224)),
                                                          transforms.ToTensor(),
                                                          transforms.Normalize(
                                                              mean=[0.456],
                                                              std=[0.224])]))

MuraTrainLoader = torch.utils.data.DataLoader(MuraTrainSet,batch_size=64,shuffle=True, num_workers=0)

MuraTestSet = MuraBodyDataset(text_file ='/content/MURA-v1.1/Elbow_Mura_Test.csv',
                              root_dir = '/content/',
                              transform = transforms.Compose([transforms.Resize((224,224)),
                                                          transforms.ToTensor(),
                                                          transforms.Normalize(
                                                              mean=[0.456],
                                                              std= [0.225])]))

MuraTestLoader = torch.utils.data.DataLoader(MuraTestSet,batch_size=64,shuffle=False, num_workers=0)

In [0]:
dense_train_loss = []
dense_train_accuracy = []
Bodypart = 'Elbow'
Path = '/content/gdrive/My Drive/MURA-v1.1'
os.path.join(Path,Bodypart)
import gc


IMG_URLS = ['/content/MURA-v1.1/train/XR_ELBOW/patient04903/study1_positive/image1.png',
            '/content/MURA-v1.1/train/XR_ELBOW/patient04904/study1_positive/image1.png',
            '/content/MURA-v1.1/train/XR_ELBOW/patient04913/study1_positive/image1.png',
            '/content/MURA-v1.1/train/XR_ELBOW/patient00894/study2_negative/image1.png',
            '/content/MURA-v1.1/train/XR_ELBOW/patient04991/study2_negative/image2.png',
            '/content/MURA-v1.1/train/XR_ELBOW/patient05023/study2_negative/image1.png']


for i in range(1,17):
  #reset model to original
  dense_net169 = models.densenet169(pretrained=True)

  for param in dense_net169.features.parameters():
      param.requires_grad = False

  dense_net169.features._modules['conv0']  = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

  #Adjust to grayscale = 1 channel. Requires gradient is True by default
  dense_net169.classifier = nn.Linear(in_features=1664, out_features=1, bias=True)

  dense_net169.to(device)

  learning = round(np.exp(random.uniform(np.log(.0001), np.log(.01))),4) #pull geometrically
  w = round(np.exp(random.uniform(np.log(3.1e-7), np.log(3.1e-5))),10) #pull geometrically


  criterion = nn.BCEWithLogitsLoss() 
  optimizer = optim.Adam(dense_net169.parameters(), lr=learning, weight_decay = w)
  
  Model = 'Model'+str(i)
  
  for epoch in range(1,21): 
      train(epoch,30, dense_train_loss,dense_train_accuracy, learning, w)
      if epoch % 5 == 0:
        net = dense_net169.to(device)
        finalconv_name = 'features'

        net.eval()

        # hook the feature extractor
        features_blobs = []

        net._modules.get(finalconv_name).register_forward_hook(hook_feature)

        # get the softmax weight
        params = list(net.parameters())
        weight_softmax = np.squeeze(params[-2].data.cpu().numpy())
        
        for j in range(len(IMG_URLS)):
          IMG_URL = IMG_URLS[j]

          img_pil = Image.open(IMG_URL)
          img_pil = img_pil.convert(mode='L')
          image_path = os.path.join(Path,Bodypart,Model,'Images','test.jpg')
          img_pil.save(image_path)

          img_tensor = preprocess(img_pil)
          img_variable = Variable(img_tensor.unsqueeze(0))
          logit = net(img_variable.cuda())
          h_x = torch.sigmoid(logit).data.squeeze()
          probs, idx = h_x.sort(0, True)
          probs = np.array([probs.cpu().numpy()])
          idx = np.array([idx.cpu().numpy()])
          #print([probs])
          #print([idx])

          CAMs = returnCAM(features_blobs[0], weight_softmax, [idx[0]])

          # render the CAM and output
          #print('output CAM.jpg for the top1 prediction: %s'%classes[idx[0]])
          img = cv2.imread(image_path,0)
          img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,11,2)
          img = np.float32(cv2.resize(img, (224, 224)))/2
          img = np.reshape(img,(224,224,1))
          height, width,_ = img.shape
          heatmap = cv2.applyColorMap(cv2.resize(CAMs[0],(width, height)), cv2.COLORMAP_WINTER)
          result = heatmap * 0.95 + img * 0.5
          image_num = 'CAM'+ str(j)+'_'+str(epoch)+'.jpg'
          #print(image_num)
          image_path = os.path.join(Path,Bodypart,Model,'Images',image_num)
          cv2.imwrite(image_path, result)
          gc.collect()
        
  Model_name = Model + '.pth'
  Model_Path = os.path.join(Path,Bodypart,Model,Model_name)
  torch.save(dense_net169.state_dict(), Model_Path)
  print('Model Completed: ', Model)

  dense_train_loss_df = pd.DataFrame(dense_train_loss,
                                    columns =['learning_rate', 'weight_decay', 'epoch', 'train_loss'] )
  dense_train_accuracy_df = pd.DataFrame(dense_train_accuracy,
                                        columns = ['learning_rate', 'weight_decay', 'epoch', 'TP', 'TN', 'FP', 'FN',
                                                   'accuracy', 'precision', 'recall', 'f1_score'])

  dense_train_loss_df.to_csv(os.path.join(Path,Bodypart,'loss.csv'))
  dense_train_accuracy_df.to_csv(os.path.join(Path,Bodypart,'accuracy.csv'))

    
    

Train Epoch:5  Accuracy: 0.60   Average Loss: 0.64  Precision: 0.63   F1 Score: 0.17 Kappa: 0.07

Train Epoch:10  Accuracy: 0.63   Average Loss: 0.62  Precision: 0.64   F1 Score: 0.29 Kappa: 0.13

Train Epoch:15  Accuracy: 0.67   Average Loss: 0.60  Precision: 0.74   F1 Score: 0.47 Kappa: 0.27

Train Epoch:20  Accuracy: 0.68   Average Loss: 0.59  Precision: 0.70   F1 Score: 0.52 Kappa: 0.30

Model Completed:  Model1
Train Epoch:5  Accuracy: 0.67   Average Loss: 0.60  Precision: 0.68   F1 Score: 0.45 Kappa: 0.25

Train Epoch:10  Accuracy: 0.72   Average Loss: 0.55  Precision: 0.75   F1 Score: 0.55 Kappa: 0.37

Train Epoch:15  Accuracy: 0.73   Average Loss: 0.53  Precision: 0.75   F1 Score: 0.63 Kappa: 0.43

Train Epoch:20  Accuracy: 0.75   Average Loss: 0.52  Precision: 0.76   F1 Score: 0.65 Kappa: 0.46

Model Completed:  Model2
Train Epoch:5  Accuracy: 0.69   Average Loss: 0.60  Precision: 0.72   F1 Score: 0.55 Kappa: 0.33

Train Epoch:10  Accuracy: 0.71   Average Loss: 0.55  Precision

**Shoulder**

In [0]:
MuraTrainSet = MuraBodyDataset(text_file ='/content/MURA-v1.1/Shoulder_Mura.csv',
                               root_dir = '/content/',
                               transform = transforms.Compose([transforms.Resize((224,224)),
                                                          transforms.ToTensor(),
                                                          transforms.Normalize(
                                                              mean=[0.456],
                                                              std=[0.224])]))

MuraTrainLoader = torch.utils.data.DataLoader(MuraTrainSet,batch_size=64,shuffle=True, num_workers=0)

MuraTestSet = MuraBodyDataset(text_file ='/content/MURA-v1.1/Shoulder_Mura_test.csv',
                              root_dir = '/content/',
                              transform = transforms.Compose([transforms.Resize((224,224)),
                                                          transforms.ToTensor(),
                                                          transforms.Normalize(
                                                              mean=[0.456],
                                                              std= [0.225])]))

MuraTestLoader = torch.utils.data.DataLoader(MuraTestSet,batch_size=64,shuffle=False, num_workers=0)

In [0]:
dense_train_loss = []
dense_train_accuracy = []
Bodypart = 'Shoulder'
Path = '/content/gdrive/My Drive/MURA-v1.1'
os.path.join(Path,Bodypart)
import gc


IMG_URLS = ['/content/MURA-v1.1/train/XR_SHOULDER/patient01277/study1_positive/image4.png',
            '/content/MURA-v1.1/train/XR_SHOULDER/patient01278/study1_positive/image1.png',
            '/content/MURA-v1.1/train/XR_SHOULDER/patient01363/study1_positive/image3.png',
            '/content/MURA-v1.1/train/XR_SHOULDER/patient01405/study2_negative/image4.png',
            '/content/MURA-v1.1/train/XR_SHOULDER/patient01412/study2_negative/image1.png',
            '/content/MURA-v1.1/train/XR_SHOULDER/patient01454/study1_negative/image4.png']


for i in range(1,17):
  #reset model to original
  dense_net169 = models.densenet169(pretrained=True)

  for param in dense_net169.features.parameters():
      param.requires_grad = False

  dense_net169.features._modules['conv0']  = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

  #Adjust to grayscale = 1 channel. Requires gradient is True by default
  dense_net169.classifier = nn.Linear(in_features=1664, out_features=1, bias=True)

  dense_net169.to(device)

  learning = round(np.exp(random.uniform(np.log(.0001), np.log(.01))),4) #pull geometrically
  w = round(np.exp(random.uniform(np.log(3.1e-7), np.log(3.1e-5))),10) #pull geometrically


  criterion = nn.BCEWithLogitsLoss() 
  optimizer = optim.Adam(dense_net169.parameters(), lr=learning, weight_decay = w)
  
  Model = 'Model'+str(i)
  
  for epoch in range(1,21): 
      train(epoch,30, dense_train_loss,dense_train_accuracy, learning, w)
      if epoch % 5 == 0:
        net = dense_net169.to(device)
        finalconv_name = 'features'

        net.eval()

        # hook the feature extractor
        features_blobs = []

        net._modules.get(finalconv_name).register_forward_hook(hook_feature)

        # get the softmax weight
        params = list(net.parameters())
        weight_softmax = np.squeeze(params[-2].data.cpu().numpy())
        
        for j in range(len(IMG_URLS)):
          IMG_URL = IMG_URLS[j]

          img_pil = Image.open(IMG_URL)
          img_pil = img_pil.convert(mode='L')
          image_path = os.path.join(Path,Bodypart,Model,'Images','test.jpg')
          img_pil.save(image_path)

          img_tensor = preprocess(img_pil)
          img_variable = Variable(img_tensor.unsqueeze(0))
          logit = net(img_variable.cuda())
          h_x = torch.sigmoid(logit).data.squeeze()
          probs, idx = h_x.sort(0, True)
          probs = np.array([probs.cpu().numpy()])
          idx = np.array([idx.cpu().numpy()])
          #print([probs])
          #print([idx])

          CAMs = returnCAM(features_blobs[0], weight_softmax, [idx[0]])

          # render the CAM and output
          #print('output CAM.jpg for the top1 prediction: %s'%classes[idx[0]])
          img = cv2.imread(image_path,0)
          img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,11,2)
          img = np.float32(cv2.resize(img, (224, 224)))/2
          img = np.reshape(img,(224,224,1))
          height, width,_ = img.shape
          heatmap = cv2.applyColorMap(cv2.resize(CAMs[0],(width, height)), cv2.COLORMAP_WINTER)
          result = heatmap * 0.95 + img * 0.5
          image_num = 'CAM'+ str(j)+'_'+str(epoch)+'.jpg'
          #print(image_num)
          image_path = os.path.join(Path,Bodypart,Model,'Images',image_num)
          cv2.imwrite(image_path, result)
          gc.collect()
        
  Model_name = Model + '.pth'
  Model_Path = os.path.join(Path,Bodypart,Model,Model_name)
  torch.save(dense_net169.state_dict(), Model_Path)
  print('Model Completed: ', Model)

  dense_train_loss_df = pd.DataFrame(dense_train_loss,
                                    columns =['learning_rate', 'weight_decay', 'epoch', 'train_loss'] )
  dense_train_accuracy_df = pd.DataFrame(dense_train_accuracy,
                                        columns = ['learning_rate', 'weight_decay', 'epoch', 'TP', 'TN', 'FP', 'FN',
                                                   'accuracy', 'precision', 'recall', 'f1_score'])

  dense_train_loss_df.to_csv(os.path.join(Path,Bodypart,'loss.csv'))
  dense_train_accuracy_df.to_csv(os.path.join(Path,Bodypart,'accuracy.csv'))

    
    

Downloading: "https://download.pytorch.org/models/densenet169-b2777c0a.pth" to /root/.torch/models/densenet169-b2777c0a.pth
57365526it [00:00, 60232780.54it/s]


Train Epoch:5  Accuracy: 0.64   Average Loss: 0.62  Precision: 0.64   F1 Score: 0.63 Kappa: 0.28

Train Epoch:10  Accuracy: 0.68   Average Loss: 0.58  Precision: 0.69   F1 Score: 0.68 Kappa: 0.37

Train Epoch:15  Accuracy: 0.69   Average Loss: 0.57  Precision: 0.68   F1 Score: 0.69 Kappa: 0.39

Train Epoch:20  Accuracy: 0.71   Average Loss: 0.56  Precision: 0.70   F1 Score: 0.70 Kappa: 0.42

Model Completed:  Model1
Train Epoch:5  Accuracy: 0.66   Average Loss: 0.61  Precision: 0.67   F1 Score: 0.66 Kappa: 0.32

Train Epoch:10  Accuracy: 0.68   Average Loss: 0.58  Precision: 0.67   F1 Score: 0.67 Kappa: 0.35

Train Epoch:15  Accuracy: 0.70   Average Loss: 0.56  Precision: 0.69   F1 Score: 0.70 Kappa: 0.39

Train Epoch:20  Accuracy: 0.72   Average Loss: 0.54  Precision: 0.73   F1 Score: 0.72 Kappa: 0.45

Model Completed:  Model2
Train Epoch:5  Accuracy: 0.67   Average Loss: 0.59  Precision: 0.66   F1 Score: 0.64 Kappa: 0.33

Train Epoch:10  Accuracy: 0.68   Average Loss: 0.58  Precision

**Hand**

In [0]:
MuraTrainSet = MuraBodyDataset(text_file ='/content/MURA-v1.1/Hand_Mura.csv',
                               root_dir = '/content/',
                               transform = transforms.Compose([transforms.Resize((224,224)),
                                                          transforms.ToTensor(),
                                                          transforms.Normalize(
                                                              mean=[0.456],
                                                              std=[0.224])]))

MuraTrainLoader = torch.utils.data.DataLoader(MuraTrainSet,batch_size=64,shuffle=True, num_workers=0)

MuraTestSet = MuraBodyDataset(text_file ='/content/MURA-v1.1/Hand_Mura_Test.csv',
                              root_dir = '/content/',
                              transform = transforms.Compose([transforms.Resize((224,224)),
                                                          transforms.ToTensor(),
                                                          transforms.Normalize(
                                                              mean=[0.456],
                                                              std= [0.225])]))

MuraTestLoader = torch.utils.data.DataLoader(MuraTestSet,batch_size=64,shuffle=False, num_workers=0)

In [0]:
dense_train_loss = []
dense_train_accuracy = []
Bodypart = 'Hand'
Path = '/content/gdrive/My Drive/MURA-v1.1'
os.path.join(Path,Bodypart)
import gc


IMG_URLS = ['/content/MURA-v1.1/train/XR_HAND/patient09740/study1_positive/image3.png',
            '/content/MURA-v1.1/train/XR_HAND/patient03009/study1_positive/image1.png',
            '/content/MURA-v1.1/train/XR_HAND/patient03959/study1_positive/image1.png',
            '/content/MURA-v1.1/train/XR_HAND/patient09740/study2_negative/image1.png',
            '/content/MURA-v1.1/train/XR_HAND/patient00218/study2_negative/image2.png',
            '/content/MURA-v1.1/train/XR_HAND/patient09944/study2_negative/image1.png']


for i in range(1,17):
  #reset model to original
  dense_net169 = models.densenet169(pretrained=True)

  for param in dense_net169.features.parameters():
      param.requires_grad = False

  dense_net169.features._modules['conv0']  = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

  #Adjust to grayscale = 1 channel. Requires gradient is True by default
  dense_net169.classifier = nn.Linear(in_features=1664, out_features=1, bias=True)

  dense_net169.to(device)

  learning = round(np.exp(random.uniform(np.log(.0001), np.log(.01))),4) #pull geometrically
  w = round(np.exp(random.uniform(np.log(3.1e-7), np.log(3.1e-5))),10) #pull geometrically


  criterion = nn.BCEWithLogitsLoss() 
  optimizer = optim.Adam(dense_net169.parameters(), lr=learning, weight_decay = w)
  
  Model = 'Model'+str(i)
  
  for epoch in range(1,21): 
      train(epoch,30, dense_train_loss,dense_train_accuracy, learning, w)
      if epoch % 5 == 0:
        net = dense_net169.to(device)
        finalconv_name = 'features'

        net.eval()

        # hook the feature extractor
        features_blobs = []

        net._modules.get(finalconv_name).register_forward_hook(hook_feature)

        # get the softmax weight
        params = list(net.parameters())
        weight_softmax = np.squeeze(params[-2].data.cpu().numpy())
        
        for j in range(len(IMG_URLS)):
          IMG_URL = IMG_URLS[j]

          img_pil = Image.open(IMG_URL)
          img_pil = img_pil.convert(mode='L')
          image_path = os.path.join(Path,Bodypart,Model,'Images','test.jpg')
          img_pil.save(image_path)

          img_tensor = preprocess(img_pil)
          img_variable = Variable(img_tensor.unsqueeze(0))
          logit = net(img_variable.cuda())
          h_x = torch.sigmoid(logit).data.squeeze()
          probs, idx = h_x.sort(0, True)
          probs = np.array([probs.cpu().numpy()])
          idx = np.array([idx.cpu().numpy()])
          #print([probs])
          #print([idx])

          CAMs = returnCAM(features_blobs[0], weight_softmax, [idx[0]])

          # render the CAM and output
          #print('output CAM.jpg for the top1 prediction: %s'%classes[idx[0]])
          img = cv2.imread(image_path,0)
          img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,11,2)
          img = np.float32(cv2.resize(img, (224, 224)))/2
          img = np.reshape(img,(224,224,1))
          height, width,_ = img.shape
          heatmap = cv2.applyColorMap(cv2.resize(CAMs[0],(width, height)), cv2.COLORMAP_WINTER)
          result = heatmap * 0.95 + img * 0.5
          image_num = 'CAM'+ str(j)+'_'+str(epoch)+'.jpg'
          #print(image_num)
          image_path = os.path.join(Path,Bodypart,Model,'Images',image_num)
          cv2.imwrite(image_path, result)
          gc.collect()
        
  Model_name = Model + '.pth'
  Model_Path = os.path.join(Path,Bodypart,Model,Model_name)
  torch.save(dense_net169.state_dict(), Model_Path)
  print('Model Completed: ', Model)

  dense_train_loss_df = pd.DataFrame(dense_train_loss,
                                    columns =['learning_rate', 'weight_decay', 'epoch', 'train_loss'] )
  dense_train_accuracy_df = pd.DataFrame(dense_train_accuracy,
                                        columns = ['learning_rate', 'weight_decay', 'epoch', 'TP', 'TN', 'FP', 'FN',
                                                   'accuracy', 'precision', 'recall', 'f1_score'])

  dense_train_loss_df.to_csv(os.path.join(Path,Bodypart,'loss.csv'))
  dense_train_accuracy_df.to_csv(os.path.join(Path,Bodypart,'accuracy.csv'))

    
    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in true_divide


Train Epoch:5  Accuracy: 0.73   Average Loss: 0.56  Precision: nan   F1 Score: nan Kappa: 0.00



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in double_scalars


Train Epoch:10  Accuracy: 0.73   Average Loss: 0.54  Precision: 1.00   F1 Score: 0.00 Kappa: 0.00

Train Epoch:15  Accuracy: 0.73   Average Loss: 0.53  Precision: 0.57   F1 Score: 0.02 Kappa: 0.01

Train Epoch:20  Accuracy: 0.72   Average Loss: 0.53  Precision: 0.72   F1 Score: 0.08 Kappa: 0.05

Model Completed:  Model1
Train Epoch:5  Accuracy: 0.77   Average Loss: 0.50  Precision: 0.63   F1 Score: 0.34 Kappa: 0.24

Train Epoch:10  Accuracy: 0.78   Average Loss: 0.48  Precision: 0.67   F1 Score: 0.43 Kappa: 0.31

Train Epoch:15  Accuracy: 0.79   Average Loss: 0.45  Precision: 0.67   F1 Score: 0.49 Kappa: 0.37

Train Epoch:20  Accuracy: 0.81   Average Loss: 0.41  Precision: 0.68   F1 Score: 0.58 Kappa: 0.46

Model Completed:  Model2
Train Epoch:5  Accuracy: 0.72   Average Loss: 0.56  Precision: nan   F1 Score: nan Kappa: 0.00

Train Epoch:10  Accuracy: 0.74   Average Loss: 0.52  Precision: 0.83   F1 Score: 0.04 Kappa: 0.03

Train Epoch:15  Accuracy: 0.75   Average Loss: 0.51  Precision:

**Wrist**

In [0]:
MuraTrainSet = MuraBodyDataset(text_file ='/content/MURA-v1.1/Wrist_Mura.csv',
                               root_dir = '/content/',
                               transform = transforms.Compose([transforms.Resize((224,224)),
                                                          transforms.ToTensor(),
                                                          transforms.Normalize(
                                                              mean=[0.456],
                                                              std=[0.224])]))

MuraTrainLoader = torch.utils.data.DataLoader(MuraTrainSet,batch_size=64,shuffle=True, num_workers=0)

MuraTestSet = MuraBodyDataset(text_file ='/content/MURA-v1.1/Wrist_Mura_test.csv',
                              root_dir = '/content/',
                              transform = transforms.Compose([transforms.Resize((224,224)),
                                                          transforms.ToTensor(),
                                                          transforms.Normalize(
                                                              mean=[0.456],
                                                              std= [0.225])]))

MuraTestLoader = torch.utils.data.DataLoader(MuraTestSet,batch_size=64,shuffle=False, num_workers=0)

In [9]:
dense_train_loss = []
dense_train_accuracy = []
Bodypart = 'Wrist'
Path = '/content/gdrive/My Drive/MURA-v1.1'
os.path.join(Path,Bodypart)
import gc


IMG_URLS = ['/content/MURA-v1.1/train/XR_WRIST/patient06361/study1_positive/image3.png',
            '/content/MURA-v1.1/train/XR_WRIST/patient06362/study1_positive/image1.png',
            '/content/MURA-v1.1/train/XR_WRIST/patient06386/study2_negative/image2.png',
            '/content/MURA-v1.1/train/XR_WRIST/patient06396/study3_negative/image3.png',
            '/content/MURA-v1.1/train/XR_WRIST/patient06398/study1_positive/image3.png',
            '/content/MURA-v1.1/train/XR_WRIST/patient06402/study2_negative/image4.png']


for i in range(1,17):
  #reset model to original
  dense_net169 = models.densenet169(pretrained=True)

  for param in dense_net169.features.parameters():
      param.requires_grad = False

  dense_net169.features._modules['conv0']  = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

  #Adjust to grayscale = 1 channel. Requires gradient is True by default
  dense_net169.classifier = nn.Linear(in_features=1664, out_features=1, bias=True)

  dense_net169.to(device)

  learning = round(np.exp(random.uniform(np.log(.0001), np.log(.01))),4) #pull geometrically
  w = round(np.exp(random.uniform(np.log(3.1e-7), np.log(3.1e-5))),10) #pull geometrically


  criterion = nn.BCEWithLogitsLoss() 
  optimizer = optim.Adam(dense_net169.parameters(), lr=learning, weight_decay = w)
  
  Model = 'Model'+str(i)
  
  for epoch in range(1,21): 
      train(epoch,30, dense_train_loss,dense_train_accuracy, learning, w)
      if epoch % 5 == 0:
        net = dense_net169.to(device)
        finalconv_name = 'features'

        net.eval()

        # hook the feature extractor
        features_blobs = []

        net._modules.get(finalconv_name).register_forward_hook(hook_feature)

        # get the softmax weight
        params = list(net.parameters())
        weight_softmax = np.squeeze(params[-2].data.cpu().numpy())
        
        for j in range(len(IMG_URLS)):
          IMG_URL = IMG_URLS[j]

          img_pil = Image.open(IMG_URL)
          img_pil = img_pil.convert(mode='L')
          image_path = os.path.join(Path,Bodypart,Model,'Images','test.jpg')
          img_pil.save(image_path)

          img_tensor = preprocess(img_pil)
          img_variable = Variable(img_tensor.unsqueeze(0))
          logit = net(img_variable.cuda())
          h_x = torch.sigmoid(logit).data.squeeze()
          probs, idx = h_x.sort(0, True)
          probs = np.array([probs.cpu().numpy()])
          idx = np.array([idx.cpu().numpy()])
          #print([probs])
          #print([idx])

          CAMs = returnCAM(features_blobs[0], weight_softmax, [idx[0]])

          # render the CAM and output
          #print('output CAM.jpg for the top1 prediction: %s'%classes[idx[0]])
          img = cv2.imread(image_path,0)
          img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,11,2)
          img = np.float32(cv2.resize(img, (224, 224)))/2
          img = np.reshape(img,(224,224,1))
          height, width,_ = img.shape
          heatmap = cv2.applyColorMap(cv2.resize(CAMs[0],(width, height)), cv2.COLORMAP_WINTER)
          result = heatmap * 0.95 + img * 0.5
          image_num = 'CAM'+ str(j)+'_'+str(epoch)+'.jpg'
          #print(image_num)
          image_path = os.path.join(Path,Bodypart,Model,'Images',image_num)
          cv2.imwrite(image_path, result)
          gc.collect()
        
  Model_name = Model + '.pth'
  Model_Path = os.path.join(Path,Bodypart,Model,Model_name)
  torch.save(dense_net169.state_dict(), Model_Path)
  print('Model Completed: ', Model)

  dense_train_loss_df = pd.DataFrame(dense_train_loss,
                                    columns =['learning_rate', 'weight_decay', 'epoch', 'train_loss'] )
  dense_train_accuracy_df = pd.DataFrame(dense_train_accuracy,
                                        columns = ['learning_rate', 'weight_decay', 'epoch', 'TP', 'TN', 'FP', 'FN',
                                                   'accuracy', 'precision', 'recall', 'f1_score'])

  dense_train_loss_df.to_csv(os.path.join(Path,Bodypart,'loss.csv'))
  dense_train_accuracy_df.to_csv(os.path.join(Path,Bodypart,'accuracy.csv'))

    
    

Downloading: "https://download.pytorch.org/models/densenet169-b2777c0a.pth" to /root/.torch/models/densenet169-b2777c0a.pth
57365526it [00:00, 37963893.15it/s]


Train Epoch:5  Accuracy: 0.62   Average Loss: 0.62  Precision: 0.62   F1 Score: 0.33 Kappa: 0.14

Train Epoch:10  Accuracy: 0.65   Average Loss: 0.60  Precision: 0.60   F1 Score: 0.47 Kappa: 0.23

Train Epoch:15  Accuracy: 0.66   Average Loss: 0.60  Precision: 0.64   F1 Score: 0.52 Kappa: 0.27

Train Epoch:20  Accuracy: 0.69   Average Loss: 0.58  Precision: 0.67   F1 Score: 0.57 Kappa: 0.34

Model Completed:  Model1
Train Epoch:5  Accuracy: 0.66   Average Loss: 0.60  Precision: 0.61   F1 Score: 0.56 Kappa: 0.29

Train Epoch:10  Accuracy: 0.71   Average Loss: 0.56  Precision: 0.66   F1 Score: 0.60 Kappa: 0.37

Train Epoch:15  Accuracy: 0.72   Average Loss: 0.53  Precision: 0.67   F1 Score: 0.65 Kappa: 0.41

Train Epoch:20  Accuracy: 0.74   Average Loss: 0.50  Precision: 0.71   F1 Score: 0.68 Kappa: 0.47

Model Completed:  Model2
Train Epoch:5  Accuracy: 0.65   Average Loss: 0.61  Precision: 0.62   F1 Score: 0.55 Kappa: 0.27

Train Epoch:10  Accuracy: 0.69   Average Loss: 0.57  Precision

**Finger**

In [0]:
MuraTrainSet = MuraBodyDataset(text_file ='/content/MURA-v1.1/Finger_Mura.csv',
                               root_dir = '/content/',
                               transform = transforms.Compose([transforms.Resize((224,224)),
                                                          transforms.ToTensor(),
                                                          transforms.Normalize(
                                                              mean=[0.456],
                                                              std=[0.224])]))

MuraTrainLoader = torch.utils.data.DataLoader(MuraTrainSet,batch_size=64,shuffle=True, num_workers=0)

MuraTestSet = MuraBodyDataset(text_file ='/content/MURA-v1.1/Finger_Mura_test.csv',
                              root_dir = '/content/',
                              transform = transforms.Compose([transforms.Resize((224,224)),
                                                          transforms.ToTensor(),
                                                          transforms.Normalize(
                                                              mean=[0.456],
                                                              std= [0.225])]))

MuraTestLoader = torch.utils.data.DataLoader(MuraTestSet,batch_size=64,shuffle=False, num_workers=0)

In [13]:
dense_train_loss = []
dense_train_accuracy = []
Bodypart = 'Finger'
Path = '/content/gdrive/My Drive/MURA-v1.1'
os.path.join(Path,Bodypart)
import gc


IMG_URLS = ['/content/MURA-v1.1/train/XR_FINGER/patient03230/study1_positive/image5.png',
            '/content/MURA-v1.1/train/XR_FINGER/patient03230/study2_negative/image1.png',
            '/content/MURA-v1.1/train/XR_FINGER/patient03237/study1_positive/image1.png',
            '/content/MURA-v1.1/train/XR_FINGER/patient03258/study2_negative/image1.png',
            '/content/MURA-v1.1/train/XR_FINGER/patient03261/study1_positive/image3.png',
            '/content/MURA-v1.1/train/XR_FINGER/patient03269/study2_negative/image3.png']


for i in range(1,17):
  #reset model to original
  dense_net169 = models.densenet169(pretrained=True)

  for param in dense_net169.features.parameters():
      param.requires_grad = False

  dense_net169.features._modules['conv0']  = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

  #Adjust to grayscale = 1 channel. Requires gradient is True by default
  dense_net169.classifier = nn.Linear(in_features=1664, out_features=1, bias=True)

  dense_net169.to(device)

  learning = round(np.exp(random.uniform(np.log(.0001), np.log(.01))),4) #pull geometrically
  w = round(np.exp(random.uniform(np.log(3.1e-7), np.log(3.1e-5))),10) #pull geometrically


  criterion = nn.BCEWithLogitsLoss() 
  optimizer = optim.Adam(dense_net169.parameters(), lr=learning, weight_decay = w)
  
  Model = 'Model'+str(i)
  
  for epoch in range(1,21): 
      train(epoch,30, dense_train_loss,dense_train_accuracy, learning, w)
      if epoch % 5 == 0:
        net = dense_net169.to(device)
        finalconv_name = 'features'

        net.eval()

        # hook the feature extractor
        features_blobs = []

        net._modules.get(finalconv_name).register_forward_hook(hook_feature)

        # get the softmax weight
        params = list(net.parameters())
        weight_softmax = np.squeeze(params[-2].data.cpu().numpy())
        
        for j in range(len(IMG_URLS)):
          IMG_URL = IMG_URLS[j]

          img_pil = Image.open(IMG_URL)
          img_pil = img_pil.convert(mode='L')
          image_path = os.path.join(Path,Bodypart,Model,'Images','test.jpg')
          img_pil.save(image_path)

          img_tensor = preprocess(img_pil)
          img_variable = Variable(img_tensor.unsqueeze(0))
          logit = net(img_variable.cuda())
          h_x = torch.sigmoid(logit).data.squeeze()
          probs, idx = h_x.sort(0, True)
          probs = np.array([probs.cpu().numpy()])
          idx = np.array([idx.cpu().numpy()])
          #print([probs])
          #print([idx])

          CAMs = returnCAM(features_blobs[0], weight_softmax, [idx[0]])

          # render the CAM and output
          #print('output CAM.jpg for the top1 prediction: %s'%classes[idx[0]])
          img = cv2.imread(image_path,0)
          img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,11,2)
          img = np.float32(cv2.resize(img, (224, 224)))/2
          img = np.reshape(img,(224,224,1))
          height, width,_ = img.shape
          heatmap = cv2.applyColorMap(cv2.resize(CAMs[0],(width, height)), cv2.COLORMAP_WINTER)
          result = heatmap * 0.95 + img * 0.5
          image_num = 'CAM'+ str(j)+'_'+str(epoch)+'.jpg'
          #print(image_num)
          image_path = os.path.join(Path,Bodypart,Model,'Images',image_num)
          cv2.imwrite(image_path, result)
          gc.collect()
        
  Model_name = Model + '.pth'
  Model_Path = os.path.join(Path,Bodypart,Model,Model_name)
  torch.save(dense_net169.state_dict(), Model_Path)
  print('Model Completed: ', Model)

  dense_train_loss_df = pd.DataFrame(dense_train_loss,
                                    columns =['learning_rate', 'weight_decay', 'epoch', 'train_loss'] )
  dense_train_accuracy_df = pd.DataFrame(dense_train_accuracy,
                                        columns = ['learning_rate', 'weight_decay', 'epoch', 'TP', 'TN', 'FP', 'FN',
                                                   'accuracy', 'precision', 'recall', 'f1_score'])

  dense_train_loss_df.to_csv(os.path.join(Path,Bodypart,'loss.csv'))
  dense_train_accuracy_df.to_csv(os.path.join(Path,Bodypart,'accuracy.csv'))

    
    

Train Epoch:5  Accuracy: 0.71   Average Loss: 0.52  Precision: 0.63   F1 Score: 0.59 Kappa: 0.37

Train Epoch:10  Accuracy: 0.73   Average Loss: 0.51  Precision: 0.68   F1 Score: 0.64 Kappa: 0.43

Train Epoch:15  Accuracy: 0.75   Average Loss: 0.48  Precision: 0.69   F1 Score: 0.66 Kappa: 0.47

Train Epoch:20  Accuracy: 0.79   Average Loss: 0.44  Precision: 0.74   F1 Score: 0.71 Kappa: 0.54

Model Completed:  Model1
Train Epoch:5  Accuracy: 0.70   Average Loss: 0.54  Precision: 0.65   F1 Score: 0.62 Kappa: 0.38

Train Epoch:10  Accuracy: 0.74   Average Loss: 0.52  Precision: 0.68   F1 Score: 0.64 Kappa: 0.43

Train Epoch:15  Accuracy: 0.76   Average Loss: 0.48  Precision: 0.69   F1 Score: 0.67 Kappa: 0.48

Train Epoch:20  Accuracy: 0.77   Average Loss: 0.46  Precision: 0.72   F1 Score: 0.72 Kappa: 0.53

Model Completed:  Model2
Train Epoch:5  Accuracy: 0.69   Average Loss: 0.55  Precision: 0.62   F1 Score: 0.59 Kappa: 0.35

Train Epoch:10  Accuracy: 0.72   Average Loss: 0.51  Precision